In [35]:
import numpy as np
from typing import List, Tuple

In [2]:
def parse_input(input_file: str) -> np.array:
    with open(input_file, "r") as file:
        char_array = None

        for line in file:
            line_chars = list(line.strip()) 
            line_array = np.array(line_chars)

            # first line: assignment
            if char_array is None:
                char_array = line_array
            # 2nd to nth line: append to existing 2d-array
            else:
                char_array = np.vstack((char_array, line_array))

    #print(char_array)
    return char_array

In [3]:
parse_input('example.txt')

array([['7', '-', 'F', '7', '-'],
       ['.', 'F', 'J', '|', '7'],
       ['S', 'J', 'L', 'L', '7'],
       ['|', 'F', '-', '-', 'J'],
       ['L', 'J', '.', 'L', 'J']], dtype='<U1')

In [37]:
# | is a vertical pipe connecting north and south.
# - is a horizontal pipe connecting east and west.
# L is a 90-degree bend connecting north and east.
# J is a 90-degree bend connecting north and west.
# 7 is a 90-degree bend connecting south and west.
# F is a 90-degree bend connecting south and east.

In [165]:
def get_next_direction(pipe_array: np.array, entry_direction: str, row_index: int, col_index: int) -> Tuple[int, int]:
    current_pipe = pipe_array[row_index, col_index]
    #print(f"Action, current_pipe: {current_pipe}, entry_direction: {entry_direction}, current_row_index: {row_index}, current_col_index: {col_index}")
    if current_pipe == "|" and entry_direction == "up":
        return "up"
    elif current_pipe == "|" and entry_direction == "down":
        return "down"
    elif current_pipe == "-" and entry_direction == "right":
        return "right"
    elif current_pipe == "-" and entry_direction == "left":
        return "left"
    elif current_pipe == "L" and entry_direction == "down":
        return "right"
    elif current_pipe == "L" and entry_direction == "left":
        return "up"
    elif current_pipe == "J" and entry_direction == "right":
        return "up"
    elif current_pipe == "J" and entry_direction == "down":
        return "left"
    elif current_pipe == "7" and entry_direction == "up":
        return "left"
    elif current_pipe == "7" and entry_direction == "right":
        return "down"
    elif current_pipe == "F" and entry_direction == "up":
        return "right"
    elif current_pipe == "F" and entry_direction == "left":
        return "down"
    else:
        print(f"Error, current_pipe: {current_pipe}, entry_direction: {entry_direction}, current_row_index: {row_index}, current_col_index: {col_index}")

In [166]:
def solve_part1(input_file: str) -> int:
    pipe_array = parse_input(input_file)
    pipe_array = np.pad(pipe_array, pad_width=1, constant_values='.')
    #print(pipe_array)
    count_array = np.zeros(pipe_array.shape, dtype=int) - 1
    
    row_indices, col_indices = np.where(pipe_array == 'S')
    start_location = np.array([row_indices[0], col_indices[0]])

    count_array[start_location[0], start_location[1]] = 0

    move_difference = {
        "up": (-1, 0),
        "down": (1, 0),
        "left": (0, -1),
        "right": (0, 1)
    }

    # get neighbor field of S (look at this to find out if your S is also a "F") adjust first direction to your input
    # start_field = pipe_array[start_location[0]-1:start_location[0]+2, start_location[1]-1:start_location[1]+2]
    # print(start_field)
    # start field in my example is "F", so I start by walking "right". 
    # You could also walk down I suppose and walk the circle in the other direction
    current_location = start_location + move_difference["right"]
    current_direction = "right"
    current_count = 1
    count_array[current_location[0], current_location[1]] = current_count
    # traverse loop
    while pipe_array[current_location[0], current_location[1]] != "S":
        current_direction = get_next_direction(pipe_array, current_direction, current_location[0], current_location[1])
        current_location = current_location + move_difference[current_direction]
        current_count += 1
        count_array[current_location[0], current_location[1]] = current_count

    print("Loop length", current_count)
    return current_count//2

In [167]:
solve_part1('input.txt')

Loop length 14194


7097

In [172]:
def solve_part2(input_file: str) -> int:
    pipe_array = parse_input(input_file)
    pipe_array = np.pad(pipe_array, pad_width=1, constant_values='.')
    #print(pipe_array)
    mark_array = np.full(pipe_array.shape, ".", dtype=str)
    
    row_indices, col_indices = np.where(pipe_array == 'S')
    start_location = np.array([row_indices[0], col_indices[0]])

    # mark all pipes of the main loop with "M"
    mark_array[start_location[0], start_location[1]] = "M"
    

    move_difference = {
        "up": (-1, 0),
        "down": (1, 0),
        "left": (0, -1),
        "right": (0, 1)
    }

    # get neighbor field of S (look at this to find out if your S is also a "F") adjust first direction to your input
    start_field = pipe_array[start_location[0]-1:start_location[0]+2, start_location[1]-1:start_location[1]+2]
    print(start_field)
    # start field in my example is "F", so I start by walking "right". 
    # You could also walk down I suppose and walk the circle in the other direction
    current_location = start_location + move_difference["right"]
    current_direction = "right"
    mark_array[current_location[0], current_location[1]] = "M"
    # traverse loop
    while pipe_array[current_location[0], current_location[1]] != "S":
        current_direction = get_next_direction(pipe_array, current_direction, current_location[0], current_location[1])
        current_location = current_location + move_difference[current_direction]
        mark_array[current_location[0], current_location[1]] = "M"

    print(mark_array)
    np.savetxt("mark_array.txt", mark_array, fmt="%s")
    return 0

In [173]:
solve_part2('example.txt')

[['.' '.' 'F']
 ['.' 'S' 'J']
 ['.' '|' 'F']]
[['.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' 'M' 'M' '.' '.']
 ['.' '.' 'M' 'M' 'M' '.' '.']
 ['.' 'M' 'M' '.' 'M' 'M' '.']
 ['.' 'M' 'M' 'M' 'M' 'M' '.']
 ['.' 'M' 'M' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.']]


0